# Preprocessing and Analysis of Healthy Subjects in the Gait Rehabilitation Project

- Focuses on understanding gait patterns and biomechanics in healthy individuals.
- Provides baseline data for comparison with patients undergoing rehabilitation.
- Aims to identify key metrics for evaluating gait performance and recovery progress.
- Creates a new dataset that simplifies the process of data usage, from EDA, ETL and machine learning training 

In [ ]:

# Import libraries
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_preprocessing import merge_data, time_domain_features, frequency_domain_features, gait_features, cross_limb_features

# Static variables and paths
base_dir = 'Data/Healthy'

# Preprocessing - Feature Extraction

### Data preprocessing

- Combines data from the left and right shank into a single dataframe.
- Facilitates ease of use by consolidating information for both sides.
- Ensures consistency in data structure for analysis and modeling.
- Allows for streamlined processing and comparison of gait metrics across both sides.

### Feature Extraction
- Extract relevant **Time Domain Features** like: Mean, Standard Deviation, Maximum, Minimum, Root Mean Square, Median Absolute Deviation, Range, Interquartile Range, Skewness & Kurtosis, Zero-crossing rate, Peak count / amplitude.

- Extract relevant **Frequency Domain Features** in a 30sec windows like: Dominant frequency, Spectral entropy, Gait band energy.

- Extract relevant **Gait Features**: Stride times, Stance/swing times, Asymmetry index, Symmetry ratio.

- Extract relevant **Cross Limb Features**: Left and right stride durations, stride duration difference, stride duration symmetry ratio.


In [ ]:
# Loop over each patient folder
for patient_folder in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_folder)
    
    if not os.path.isdir(patient_path):
        continue 

    print(f"\nProcessing {patient_folder}...")

    # 1. Merge gyroscope data for the patient
    left_path = os.path.join(patient_path, 'LeftShank-Gyroscope.csv')
    right_path = os.path.join(patient_path, 'RightShank-Gyroscope.csv')
    merge_data(patient_path + '/', left_path, right_path, 'gyroscope')

    # 2. Run all feature extraction functions 
    time_domain_features(patient_path + '/', 'gyroscope')
    frequency_domain_features(patient_path + '/', 'gyroscope')
    gait_features(patient_path + '/', 'gyroscope')
    cross_limb_features(patient_path + '/', 'gyroscope')

    print(f"Finished {patient_folder}")